# Wrangle and Analyze Data Project Details 
1. `wrangle_act.ipynb`: code for gathering, assessing, cleaning, analyzing, and visualizing data
2. `wrangle_report.pdf` or `wrangle_report.html`: `300-600` word documentation for data wrangling steps: gather, assess, and clean 
2. `act_report.pdf` or `act_report.html`: `250` word minimum documentation of analysis and insights into final data
3. `twitter_archive_enhanced.csv`: file as given
4. `image_predictions.tsv`: file downloaded programmatically
5. `tweet_json.txt`: file constructed via API
6. `twitter_archive_master.csv`: combined and cleaned data
7. any additional files
8. At least three `3` insights and one `1` visualization must be assessed. 


## Part I: Gather

In [1]:
import pandas as pd
import requests
import os
import logging
import sys
import json
import datetime
import numpy as np
# global logger level is configured in main()
Logger = None

#### OS, File I/O, Requests

In [19]:
folder_name = 'tweet_image_predictions'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open(os.path.join(folder_name, url.split('/')[-1]), mode='wb') as file:
    file.write(response.content)

#### Twitter API, Tweepy, File I/O 

1. Query all of the tweet IDs in the WeRateDogs Twitter archive, printing out each tweet ID after it was queried.
2. Set the wait_on_rate_limit and wait_on_rate_limit_notify parameters to True in the tweepy.api class.
3. Tweet data is stored in JSON format by Twitter. 
4. Set the tweet_mode parameter to 'extended' in the get_status call, i.e., api.get_status(tweet_id, tweet_mode='extended').
- You only want original ratings (no retweets) that have images. 
- Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.

In [21]:
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer

consumer_key = 'API_KEY'
consumer_secret = 'SECRET'
access_token = 'TOKEN'
access_secret = 'SECRET'
    
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [ ]:
tweet_ids = df.tweet_id.values
len(tweet_ids)

count = 0
fails_dict = {}
start = timer()
with open('tweet_json.txt', 'w') as outfile:
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Got thru!")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Did not get thru.")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

In [3]:
with open('tweet_json.txt') as file:
    tweet_json_list = []
    for line in file:
        tweet_json_list.append(json.loads(line)) # cite 1

In [4]:
tweet_json_list[7]

{'created_at': 'Fri Jul 28 00:22:40 +0000 2017',
 'id': 890729181411237888,
 'id_str': '890729181411237888',
 'full_text': "When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq",
 'truncated': False,
 'display_text_range': [0, 118],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 890729118844600320,
    'id_str': '890729118844600320',
    'indices': [119, 142],
    'media_url': 'http://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DFyBag_UQAAhhBC.jpg',
    'url': 'https://t.co/v0nONBcwxq',
    'display_url': 'pic.twitter.com/v0nONBcwxq',
    'expanded_url': 'https://twitter.com/dog_rates/status/890729181411237888/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'large': {'w': 1200, 'h': 1328, 'resize': 'fit'},
     'medium': {'w': 1084, 'h': 1200,

In [5]:
df_json = pd.DataFrame.from_records(tweet_json_list)

#### Pandas read_csv and tsv

In [6]:
df_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [7]:
df_tsv = pd.read_csv('tweet_image_predictions/image-predictions.tsv', sep='\t', header=0)

## Part II: Assess

- Assess and clean at least **eight `8` quality issues** and **two `2` tidiness issues** in this dataset.
- Cleaning includes merging individual pieces of data according to the rules of tidy data.
- The fact that the rating numerators are greater than the denominators does not need to be cleaned. 
- You do not need to gather the tweets beyond August 1st, 2017. 
- You only want ratings (no retweets) that have images from JSON Twitter API data
- Image prediction scores are in `tsv` for each dog and T/F confirms prediction accurate or not
- Dog stages are: `doggo` (big pupper and older), `floofer` (any dog with +fur), `pupper` (small doggo and younger), `puppo` (transition between pupper and doggo)

In [8]:
df_archive.sample(1)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
22,887517139158093824,NaN,NaN,2017-07-19 03:39:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I've yet to rate a Venezuelan Hover Wiener. Th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/887517139...,14,10,such,None,None,None,None


In [9]:
df_tsv.sample(3)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1240,746906459439529985,https://pbs.twimg.com/media/Cl2LdofXEAATl7x.jpg,1,traffic_light,0.470708,False,fountain,0.199776,False,space_shuttle,0.064807,False
530,676819651066732545,https://pbs.twimg.com/media/CWSL4W8WsAAE4KU.jpg,2,rain_barrel,0.625555,False,barrel,0.138383,False,Labrador_retriever,0.039465,True
1343,759047813560868866,https://pbs.twimg.com/media/Coit84_VYAEMtLi.jpg,1,Labrador_retriever,0.778546,True,bathing_cap,0.154254,False,golden_retriever,0.024972,True


In [10]:
df_json.sample(1)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
1146,None,None,Fri Apr 15 15:44:11 +0000 2016,"[0, 131]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 721001172476137472, 'id_str'...",2585,False,This is Oliver. Bath time is upon him. His fea...,None,...,NaN,NaN,NaN,NaN,626,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."


In [11]:
df_json.info() # only want ratings (not retweets) with url image

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2333 entries, 0 to 2332
Data columns (total 32 columns):
contributors                     0 non-null object
coordinates                      0 non-null object
created_at                       2333 non-null object
display_text_range               2333 non-null object
entities                         2333 non-null object
extended_entities                2061 non-null object
favorite_count                   2333 non-null int64
favorited                        2333 non-null bool
full_text                        2333 non-null object
geo                              0 non-null object
id                               2333 non-null int64
id_str                           2333 non-null object
in_reply_to_screen_name          77 non-null object
in_reply_to_status_id            77 non-null float64
in_reply_to_status_id_str        77 non-null object
in_reply_to_user_id              77 non-null float64
in_reply_to_user_id_str          77 non-null obj

In [12]:
df_tsv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [13]:
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

### Quality

**twitter-archive-enhanced.csv**: 
1. `NaN` values in `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_timestamp`
2. `None` values in `name`,  `doggo`, `floofer`, `pupper`, `puppo`
3. Single letters in `name`

**tweet_json.txt**: 
4.  `None` values in `contributors`, `coordinates`, `geo`
5. `NaN` values in `extended_entities`, `quoted_status`, `quoted_status_id`, `quoted_status_id_str`, `quoted_status_permalink`, `retweeted_status`
6. remame `id_str` to `tweet_id` for merging dataframes

**image-predictions.tsv**
7. `p1, p2, p3` have both upper and lower case strings 
8. `p1, p2, p3` have `_` between name
9. `p1, p2, p3` have non-dog names

### Tidiness


- Column headers are values, not variable names.
- Multiple variables are stored in one column.
- Variables are stored in both rows and columns.
- Multiple types of observational units are stored in the same table.
- A single observational unit is stored in multiple tables.

1. `image-predictions.tsv` columns names need to be renamed to show prediction outcome
2. all three `json`, `tsv`, `txt` DataFrames need to be merged to combine only important data joining on `tweet_id`

## Part III: Clean

In [358]:
archive_clean = df_archive.copy()
json_clean = df_json.copy()
tsv_clean = df_tsv.copy()

In [359]:
archive_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

In [360]:
json_clean.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'favorite_count', 'favorited',
       'full_text', 'geo', 'id', 'id_str', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status',
       'lang', 'place', 'possibly_sensitive', 'possibly_sensitive_appealable',
       'quoted_status', 'quoted_status_id', 'quoted_status_id_str',
       'quoted_status_permalink', 'retweet_count', 'retweeted',
       'retweeted_status', 'source', 'truncated', 'user'],
      dtype='object')

In [361]:
tsv_clean.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'],
      dtype='object')

#### drop columns, drop null values, convert to timestamp, ignore lower case names

In [362]:
archive_clean.drop(columns=['in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp', 'text', 'retweeted_status_id', 'retweeted_status_user_id','retweeted_status_timestamp', 'expanded_urls'], inplace=True)

In [363]:
json_clean.drop(columns=['contributors', 'coordinates', 'display_text_range', 'entities', 'extended_entities', 'favorited', 'geo', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place', 'possibly_sensitive', 'possibly_sensitive_appealable','quoted_status', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink', 'retweeted', 'retweeted_status', 'source', 'truncated', 'user'], inplace=True)

In [364]:
tsv_clean.drop(columns=['img_num', 'p1_conf', 'p2_conf', 'p3_conf'], inplace=True)

In [365]:
archive_clean.replace(['None'], np.nan, inplace = True) # replace None with NaN in order to dropna truthy 
archive_clean.drop(archive_clean[  (archive_clean['name'].str.len() <=3 )].index, inplace=True) # drop names <=3 

In [366]:
archive_clean.isnull().sum()

tweet_id                 0
source                   0
rating_numerator         0
rating_denominator       0
name                   745
doggo                 2084
floofer               2169
pupper                1940
puppo                 2147
dtype: int64

In [367]:
archive_clean['name'].replace('such','NaN',inplace=True) # edge case
archive_clean['name'].replace('quite','NaN',inplace=True) # edge case 

In [368]:
archive_clean.shape, json_clean.shape, tsv_clean.shape

((2177, 9), (2333, 5), (2075, 8))

In [369]:
json_clean['created_at'] = pd.to_datetime(json_clean['created_at'])

In [370]:
json_clean['created_at'] = json_clean['created_at'].map(lambda x: x.strftime('%Y%m%d%' + ' ' + '%I:%M %p') if x else "") # cite 2

#### rename columns

In [371]:
json_clean = json_clean.rename(columns={"id": "tweet_id", "created_at": "timestamp"})

In [372]:
archive_clean = archive_clean.rename(columns={"name": "dog_name"})

In [373]:
tsv_clean = tsv_clean.rename(columns={"p1": "first_dog_prediction", "p1_dog": "first_prediction_outcome", \
                         "p2": "second_dog_prediction", "p2_dog": "second_prediction_outcome", \
                          "p3": "third_dog_prediction", "p3_dog": "third_prediction_outcome"})

#### string function replace and title

In [374]:
tsv_clean['first_dog_prediction'] = tsv_clean['first_dog_prediction'].astype(str).replace(to_replace ='_', value = ' ', regex = True) 
tsv_clean['first_dog_prediction'] = tsv_clean['first_dog_prediction'].astype(str).str.title()

tsv_clean['second_dog_prediction'] = tsv_clean['second_dog_prediction'].astype(str).replace(to_replace ='_', value = ' ', regex = True) 
tsv_clean['second_dog_prediction'] = tsv_clean['second_dog_prediction'].astype(str).str.title()

tsv_clean['third_dog_prediction'] = tsv_clean['third_dog_prediction'].astype(str).replace(to_replace ='_', value = ' ', regex = True) 
tsv_clean['third_dog_prediction'] = tsv_clean['third_dog_prediction'].astype(str).str.title()

In [375]:
tsv_clean.tweet_id.dtype, json_clean.tweet_id.dtype, archive_clean.tweet_id.dtype # check join key same type 

(dtype('int64'), dtype('int64'), dtype('int64'))

In [376]:
# clean ratings: df['rating_numerator'] = df.text.str.extract('((?:\d+\.)?\d+)\/(\d+)', expand=True) #regex extracting
# # cite: https://knowledge.udacity.com/questions/49748

# replace column names with nan: 
# mask1 = twitter_clean.name.str.isupper()
# mask2 = df_1_clean.name.str.islower()
# column_name = 'name'
# df_clean.loc[(mask1 | mask2), column_name] = np.nan 
# # cite: https://knowledge.udacity.com/questions/45647

#### Merge DataFrames

In [378]:
json_clean.sample(3), tsv_clean.sample(3), archive_clean.sample(3)

In [379]:
json_clean.columns, tsv_clean.columns, archive_clean.columns

(Index(['timestamp', 'favorite_count', 'full_text', 'tweet_id',
        'retweet_count'],
       dtype='object'), Index(['tweet_id', 'jpg_url', 'first_dog_prediction',
        'first_prediction_outcome', 'second_dog_prediction',
        'second_prediction_outcome', 'third_dog_prediction',
        'third_prediction_outcome'],
       dtype='object'), Index(['tweet_id', 'source', 'rating_numerator', 'rating_denominator',
        'dog_name', 'doggo', 'floofer', 'pupper', 'puppo'],
       dtype='object'))

In [380]:
from functools import reduce
dfs = [json_clean, tsv_clean, archive_clean]
master = reduce(lambda left,right: pd.merge(left,right,on='tweet_id'), dfs) # cite 3

In [382]:
master.sample(1)

,timestamp,favorite_count,full_text,tweet_id,retweet_count,jpg_url,first_dog_prediction,first_prediction_outcome,second_dog_prediction,second_prediction_outcome,third_dog_prediction,third_prediction_outcome,source,rating_numerator,rating_denominator,dog_name,doggo,floofer,pupper,puppo
1628,20151201 04:22 AM,1961,Interesting dog here. Very large. Purple. Mani...,671544874165002241,1037,https://pbs.twimg.com/media/CVHOgDvU4AAfrXD.jpg,Feather Boa,False,Wig,False,Wool,False,"<a href=""http://twitter.com/download/iphone"" r...",6,10,NaN,NaN,NaN,NaN,NaN


In [383]:
master.columns

Index(['timestamp', 'favorite_count', 'full_text', 'tweet_id', 'retweet_count',
       'jpg_url', 'first_dog_prediction', 'first_prediction_outcome',
       'second_dog_prediction', 'second_prediction_outcome',
       'third_dog_prediction', 'third_prediction_outcome', 'source',
       'rating_numerator', 'rating_denominator', 'dog_name', 'doggo',
       'floofer', 'pupper', 'puppo'],
      dtype='object')

In [384]:
master.reindex(columns=['timestamp', 'tweet_id', 'full_text', 'jpg_url', 'source', 'dog_name', 
        'favorite_count', 'retweet_count', 'rating_numerator', 'rating_denominator',
       'first_dog_prediction', 'first_prediction_outcome','second_dog_prediction', 'second_prediction_outcome',
       'third_dog_prediction', 'third_prediction_outcome', 'doggo', 'floofer', 'pupper', 'puppo'])

,timestamp,tweet_id,full_text,jpg_url,source,dog_name,favorite_count,retweet_count,rating_numerator,rating_denominator,first_dog_prediction,first_prediction_outcome,second_dog_prediction,second_prediction_outcome,third_dog_prediction,third_prediction_outcome,doggo,floofer,pupper,puppo
0,20170801 04:23 PM,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,"<a href=""http://twitter.com/download/iphone"" r...",Phineas,37274,7983,13,10,Orange,False,Bagel,False,Banana,False,NaN,NaN,NaN,NaN
1,20170801 12:17 AM,892177421306343426,This is Tilly. She's just checking pup on you....,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,"<a href=""http://twitter.com/download/iphone"" r...",Tilly,32047,5928,13,10,Chihuahua,True,Pekinese,True,Papillon,True,NaN,NaN,NaN,NaN
2,20170731 12:18 AM,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,"<a href=""http://twitter.com/download/iphone"" r...",Archie,24147,3915,12,10,Chihuahua,True,Malamute,True,Kelpie,True,NaN,NaN,NaN,NaN
3,20170730 03:58 PM,891689557279858688,This is Darla. She commenced a snooze mid meal...,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,"<a href=""http://twitter.com/download/iphone"" r...",Darla,40588,8129,13,10,Paper Towel,False,Labrador Retriever,True,Spatula,False,NaN,NaN,NaN,NaN
4,20170729 04:00 PM,891327558926688256,This is Franklin. He would like you to stop ca...,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,"<a href=""http://twitter.com/download/iphone"" r...",Franklin,38826,8813,12,10,Basset,True,English Springer,True,German Short-Haired Pointer,True,NaN,NaN,NaN,NaN
5,20170729 12:08 AM,891087950875897856,Here we have a majestic great white breaching ...,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,"<a href=""http://twitter.com/download/iphone"" r...",NaN,19501,2931,13,10,Chesapeake Bay Retriever,True,Irish Terrier,True,Indian Elephant,False,NaN,NaN,NaN,NaN
6,20170728 12:22 AM,890729181411237888,When you watch your owner call another dog a g...,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,"<a href=""http://twitter.com/download/iphone"" r...",NaN,62862,17788,13,10,Pomeranian,True,Eskimo Dog,True,Pembroke,True,NaN,NaN,NaN,NaN
7,20170727 04:25 PM,890609185150312448,This is Zoey. She doesn't want to be one of th...,https://pbs.twimg.com/media/DFwUU__XcAEpyXI.jpg,"<a href=""http://twitter.com/download/iphone"" r...",Zoey,26842,4033,13,10,Irish Terrier,True,Irish Setter,True,Chesapeake Bay Retriever,True,NaN,NaN,NaN,NaN
8,20170726 03:59 PM,890240255349198849,This is Cassie. She is a college pup. Studying...,https://pbs.twimg.com/media/DFrEyVuW0AAO3t9.jpg,"<a href=""http://twitter.com/download/iphone"" r...",Cassie,30741,6949,14,10,Pembroke,True,Cardigan,True,Chihuahua,True,doggo,NaN,NaN,NaN
9,20170726 12:31 AM,890006608113172480,This is Koda. He is a South Australian decksha...,https://pbs.twimg.com/media/DFnwSY4WAAAMliS.jpg,"<a href=""http://twitter.com/download/iphone"" r...",Koda,29576,6905,13,10,Samoyed,True,Pomeranian,True,Chow,True,NaN,NaN,NaN,NaN


#### Storing, Analyzing, and Visualizing Data

In [385]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///tweets.db')

In [387]:
master.to_sql('tweets', engine, index=False)

In [388]:
tweets = pd.read_sql('SELECT * FROM tweets', engine)

In [389]:
tweets.head(3)

,timestamp,favorite_count,full_text,tweet_id,retweet_count,jpg_url,first_dog_prediction,first_prediction_outcome,second_dog_prediction,second_prediction_outcome,third_dog_prediction,third_prediction_outcome,source,rating_numerator,rating_denominator,dog_name,doggo,floofer,pupper,puppo
0,20170801 04:23 PM,37274,This is Phineas. He's a mystical boy. Only eve...,892420643555336193,7983,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,Orange,0,Bagel,0,Banana,0,"<a href=""http://twitter.com/download/iphone"" r...",13,10,Phineas,None,None,None,None
1,20170801 12:17 AM,32047,This is Tilly. She's just checking pup on you....,892177421306343426,5928,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,Chihuahua,1,Pekinese,1,Papillon,1,"<a href=""http://twitter.com/download/iphone"" r...",13,10,Tilly,None,None,None,None
2,20170731 12:18 AM,24147,This is Archie. He is a rare Norwegian Pouncin...,891815181378084864,3915,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,Chihuahua,1,Malamute,1,Kelpie,1,"<a href=""http://twitter.com/download/iphone"" r...",12,10,Archie,None,None,None,None


> Citations:
1. https://stackoverflow.com/questions/47889565/reading-json-objects-from-text-file-into-pandas
2. https://stackoverflow.com/questions/35488036/using-lambda-and-strftime-on-dates-when-there-are-null-values-pandas
3. https://stackoverflow.com/questions/49127961/python-pandas-can-not-merge-dataframe-with-instance-of-type-class-str